In [1]:
import tensorflow as tf
import pathlib
import PIL
import numpy as np

import matplotlib
matplotlib.use("TKAgg")
import matplotlib.pyplot as plt

import torch
import torchvision.transforms as transforms
from PIL import Image
import os
import scipy

AUTOTUNE = tf.data.AUTOTUNE # prompts the tf.data runtime to tune the value dynamically at runtime

print(tf. __version__)

2023-04-05 12:03:19.695923: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-05 12:03:19.730870: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


2.11.1


In [3]:
batch_size = 32
img_height = 28
img_width = 28

def create_dataset_from_path(path):

    # function that converts a file path to an (img, label) pair
    def get_label(file_path):
      # Convert the path to a list of path components
      parts = tf.strings.split(file_path, os.path.sep)
      # The second to last is the class-directory
      one_hot = parts[-2] == class_names
      # Integer encode the label
      return tf.argmax(one_hot)

    def decode_img(img):
      # Convert the compressed string to a 3D uint8 tensor
      img = tf.io.decode_jpeg(img, channels=1)
      # Resize the image to the desired size
      return tf.image.resize(img, [img_height, img_width])

    def process_path(file_path):
      label = get_label(file_path)
      # Load the raw data from the file as a string
      img = tf.io.read_file(file_path)
      img = decode_img(img)
      return img, label

    # Make the imgs in to batches
    def configure_for_performance(ds):
      ds = ds.cache()
      ds = ds.shuffle(buffer_size=1000)
      ds = ds.batch(batch_size)
      ds = ds.prefetch(buffer_size=AUTOTUNE)
      return ds

    data_dir = pathlib.Path(path) # for pycharm
    #data_dir = pathlib.Path('mnist_png/training') # for the jupyter notebook

    print(path+'/*/*')

    # Shuffle might be better set to False but this seems better for now
    list_ds = tf.data.Dataset.list_files((path+'/*/*'), shuffle=True)

    image_count = len(list_ds)
    class_names = np.array(sorted([item.name for item in data_dir.glob('*')]))
    print(class_names)
    val_size = int(image_count * 0.2)
    train_ds = list_ds.skip(val_size) # skips x initial elements from this dataset.
    val_ds = list_ds.take(val_size)
    print(tf.data.experimental.cardinality(train_ds).numpy())
    print(tf.data.experimental.cardinality(val_ds).numpy())


    # Set `num_parallel_calls` so multiple images are loaded/processed in parallel.
    train_ds = train_ds.map(process_path, num_parallel_calls=AUTOTUNE)
    val_ds = val_ds.map(process_path, num_parallel_calls=AUTOTUNE)

    normalization_layer = tf.keras.layers.Rescaling(1./255)

    normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
    # normalized_ds  -> range goes from 0 to 1
    # train_ds  -> range goes from 0 to

    val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))

    normalized_ds = configure_for_performance(normalized_ds)
    val_ds = configure_for_performance(val_ds)

    print(normalized_ds)
    return normalized_ds, val_ds, class_names


def create_dataset_from_img(path):

    def decode_img(img):
      # Convert the compressed string to a 3D uint8 tensor
      img = tf.io.decode_jpeg(img, channels=1)
      # Resize the image to the desired size
      return tf.image.resize(img, [img_height, img_width])

    def process_path(file_path):
      label = tf.strings.split(file_path, os.path.sep)[-2]
      #print(label)
      # Load the raw data from the file as a string
      img = tf.io.read_file(file_path)
      img = decode_img(img)
      return img, label


    # Make the imgs in to batches
    def configure_for_performance(ds):
      ds = ds.cache()
      ds = ds.shuffle(buffer_size=1000)
      ds = ds.batch(1)
      ds = ds.prefetch(buffer_size=AUTOTUNE)
      return ds

    data_dir = pathlib.Path(path) # for pycharm
    #data_dir = pathlib.Path('mnist_png/training') # for the jupyter notebook

    #print(path+'\\*\\*')


    list_ds = tf.data.Dataset.list_files(path, shuffle=True)

    image_count = len(list_ds)
    class_name = [int(tf.strings.split(path, os.path.sep)[-2])]
    #print(f"Class -> {class_name}")
    val_size = int(image_count)
    val_ds = list_ds.take(val_size)
    #print(tf.data.experimental.cardinality(val_ds).numpy())



    # Set `num_parallel_calls` so multiple images are loaded/processed in parallel.
    val_ds = val_ds.map(process_path, num_parallel_calls=AUTOTUNE)

    normalization_layer = tf.keras.layers.Rescaling(1./255)
    val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))

    val_ds = configure_for_performance(val_ds)


    return val_ds, class_name

In [4]:
os.getcwd()
#os.chdir('D:\AI_Projects\Classifier_AI')
os.getcwd()

'/home/jupyter-gmateus@ilof.tech/Classifier_AI'

In [5]:
normalized_ds, val_ds, class_names = create_dataset_from_path('mnist_png/training')

mnist_png/training/*/*
['0' '1' '2' '3' '4' '5' '6' '7' '8' '9']
48000
12000
<PrefetchDataset element_spec=(TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>


2023-04-04 23:08:50.415531: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-04 23:08:50.415996: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-04 23:08:50.416049: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-04 23:08:50.416321: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the

In [6]:
# Using the normalized
for imageBatch, labelBatch in normalized_ds.take(1):
    print("Image shape: ", imageBatch.numpy()[0].shape)
    print("Label: ", labelBatch.numpy()[0])
    print("Label: ", labelBatch.numpy())
    #print(np.min(image), np.max(image))

    # f = open("demofile.txt", "a")
    # for a in range(28): # goes from 0 to 27
    #   for b in range(28):
    #     print(f'{round(float(image[b][a][0]), 1)}',end='')
    #     f.write(str(round(float(image[b][a][0]), 1)))
    #   print(f'\n')
    #   f.write('\n')
    # f.close()

    imgToBeDisplayed = imageBatch.numpy()[0]

    extraChannel = np.zeros((28,28,1))
    imgToBeDisplayed = np.concatenate((imgToBeDisplayed,extraChannel), axis=2)
    imgToBeDisplayed = np.concatenate((imgToBeDisplayed,extraChannel), axis=2)
    imgToBeDisplayed = imgToBeDisplayed * 255

    print("Image shape: ", imgToBeDisplayed.shape)
    print("Label: ", labelBatch.numpy()[0])
    # #print(f'->{image}')

    print(np.min(imageBatch[0]), np.max(imageBatch[0]))

    image_uint8 = imgToBeDisplayed.astype(np.uint8)
    img = Image.fromarray(image_uint8, 'RGB')
    #img.show()
    #img.save('Classifier_AI\\temp.png')
    img.save('temp.png')

#PIL.Image.open('Classifier_AI\\temp.png')
PIL.Image.open('temp.png')

Image shape:  (28, 28, 1)
Label:  2
Label:  [2 3 9 1 5 6 7 2 3 4 4 9 4 6 6 5 9 2 9 5 3 7 7 1 9 5 5 1 1 8 7 4]
Image shape:  (28, 28, 3)
Label:  2
0.0 1.0


2023-04-04 23:08:52.775342: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [7]:
os.remove('temp.png')

In [8]:
num_classes = 10

model = tf.keras.Sequential([
  tf.keras.layers.Conv2D(filters=15, kernel_size=(5,5), padding='Same', activation='relu', input_shape=(28, 28, 1),
                         data_format="channels_last"),
  tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),

  tf.keras.layers.Conv2D(filters=20, kernel_size=(10,10), padding='Same', activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),

  tf.keras.layers.Conv2D(filters=25, kernel_size=(15,15), padding='Same', activation='relu'),
  tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),

  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_classes)
])

In [9]:
model.compile(
  optimizer='adam',
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])


In [10]:
epochs=10

history = model.fit(
  normalized_ds,
  validation_data=val_ds,
  epochs=epochs
)



Epoch 1/10


2023-04-04 23:08:58.103373: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8500


  49/1500 [..............................] - ETA: 1s - loss: 1.4155 - accuracy: 0.5134   

2023-04-04 23:08:58.313334: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-04-04 23:08:58.313679: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-04-04 23:08:58.313699: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2023-04-04 23:08:58.314011: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-04-04 23:08:58.314046: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2023-04-04 23:08:58.322711: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged onc

1500/1500 [==============================] - 3s 1ms/step - loss: 0.1669 - accuracy: 0.9461 - val_loss: 0.0590 - val_accuracy: 0.9818
Epoch 2/10
1500/1500 [==============================] - 1s 982us/step - loss: 0.0492 - accuracy: 0.9849 - val_loss: 0.0392 - val_accuracy: 0.9878
Epoch 3/10
1500/1500 [==============================] - 1s 950us/step - loss: 0.0328 - accuracy: 0.9895 - val_loss: 0.0288 - val_accuracy: 0.9912
Epoch 4/10
1500/1500 [==============================] - 1s 918us/step - loss: 0.0236 - accuracy: 0.9924 - val_loss: 0.0190 - val_accuracy: 0.9942
Epoch 5/10
1500/1500 [==============================] - 1s 917us/step - loss: 0.0188 - accuracy: 0.9941 - val_loss: 0.0178 - val_accuracy: 0.9943
Epoch 6/10
1500/1500 [==============================] - 1s 970us/step - loss: 0.0166 - accuracy: 0.9951 - val_loss: 0.0208 - val_accuracy: 0.9942
Epoch 7/10
1500/1500 [==============================] - 1s 976us/step - loss: 0.0125 - accuracy: 0.9963 - val_loss: 0.0179 - val_accuracy

In [11]:
model.save('saved_models')

model.summary()

INFO:tensorflow:Assets written to: saved_models/assets


INFO:tensorflow:Assets written to: saved_models/assets


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 15)        390       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 15)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 20)        30020     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 7, 7, 20)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 7, 7, 25)          112525    
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 3, 3, 25)         0

In [12]:
model = tf.keras.models.load_model('saved_models')

In [ ]:
#tf.keras.utils.plot_model(model, to_file='model.png',
#                          show_shapes=True, show_layer_activations=True)

In [13]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)



In [14]:
plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)

plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

ImportError: Cannot load backend 'TkAgg' which requires the 'tk' interactive framework, as 'headless' is currently running

In [15]:
def test_model(img_path):

    test_val_ds, test_class_names = create_dataset_from_img(img_path)

    predictions = model.predict(test_val_ds)
    score = tf.nn.softmax(predictions[0])

    answerBool = (int(class_names[np.argmax(score)]) == test_class_names[0])

    print(
        "[{}] This image most likely belongs to {} with a {:.2f} percent confidence."
        .format(answerBool, class_names[np.argmax(score)], 100 * np.max(score))
    )

    for imageBatch, labelBatch in test_val_ds.take(1):
        imgToBeDisplayed = imageBatch.numpy()[0]

        extraChannel = np.zeros((28,28,1))
        imgToBeDisplayed = np.concatenate((imgToBeDisplayed,extraChannel), axis=2)
        imgToBeDisplayed = np.concatenate((imgToBeDisplayed,extraChannel), axis=2)
        imgToBeDisplayed = imgToBeDisplayed * 255

        image_uint8 = imgToBeDisplayed.astype(np.uint8)
        img = Image.fromarray(image_uint8, 'RGB')
        #img.show()
        img.save('temp.png')


In [21]:
#test_model('D:\\AI_Projects\\Classifier_AI\\mnist_png\\testing\\6\\156.png')
test_model('/home/jupyter-gmateus@ilof.tech/Classifier_AI/mnist_png/testing/6/156.png')
PIL.Image.open('temp.png')

1/1 [==============================] - 0s 49ms/step
[True] This image most likely belongs to 6 with a 100.00 percent confidence.


In [16]:
convert_tensor = transforms.ToTensor()

img_tensor = convert_tensor(img)

print(img_tensor.shape)
testing_ds = tf.keras.utils.image_dataset_from_directory(
    pathlib.Path('mnist_png/testing'),
    seed=123,
    image_size=(img_height, img_width),
    batch_size=1,
    color_mode='grayscale'
)

torch.Size([3, 28, 28])
Found 10000 files belonging to 10 classes.


In [17]:
model.predict(testing_ds)

10000/10000 [==============================] - 3s 323us/step


array([[-4664.8027 ,   -64.41077,  9443.199  , ...,   -79.61272,
        -5259.7104 , -7442.3696 ],
       [-2168.943  ,  -323.61227,  4686.1377 , ...,   544.9698 ,
        -1086.2988 , -1132.8331 ],
       [-2628.83   ,   559.7561 ,   354.7001 , ...,  6772.3228 ,
        -3000.367  , -1558.6617 ],
       ...,
       [ -696.9902 ,  -187.53683,  -520.27124, ...,  2715.7966 ,
         -421.96918,   264.89276],
       [ 3337.8408 , -2763.7156 ,  -627.5956 , ...,  -133.08769,
         -581.16895,  1596.3546 ],
       [-2239.03   , -1153.796  , -1198.9889 , ..., -1286.4437 ,
          327.44672, -1133.6217 ]], dtype=float32)